In [1]:
import pandas as pd
import numpy as np
import random
import os
print(os.getcwd())


C:\Users\Dango\Documents\Fall2022\Data Science


In [2]:
df = pd.read_csv("filtered_data.csv")
print(df.shape)
print(df.info())
print(df.head())
#(6129757, 9)

(6129757, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6129757 entries, 0 to 6129756
Data columns (total 9 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Registration State  object
 1   Plate Type          object
 2   Violation Code      int64 
 3   Vehicle Body Type   object
 4   Vehicle Make        object
 5   Violation Precinct  int64 
 6   Violation Time      object
 7   Violation County    object
 8   Vehicle Color       object
dtypes: int64(2), object(7)
memory usage: 420.9+ MB
None
  Registration State Plate Type  Violation Code Vehicle Body Type  \
0                 NY        PAS              67               SDN   
1                 NY        PAS              51              SUBN   
2                 NJ        PAS              63               SDN   
3                 NY        PAS              63              SUBN   
4                 NJ        PAS              63               SDN   

  Vehicle Make  Violation Precinct Violation Time V

In [3]:
# question we are answering
"""
We will be building a model to see if we could predict violation 
code based on vehicle descriptions and location.
Hence, this brings the awareness where people could try to be more careful if they match description
"""

'\nWe will be building a model to see if we could predict violation \ncode based on vehicle descriptions and location.\nHence, this brings the awareness where people could try to be more careful if they match description\n'

In [4]:
# observe data
print(df.isnull().sum())
# there are only null values in the following columns
# Vehicle Body Type
# Vehicle Make
# Violation Time
# Violation County
# Vehicle Color

Registration State         0
Plate Type                 0
Violation Code             0
Vehicle Body Type      15645
Vehicle Make            5485
Violation Precinct         0
Violation Time            66
Violation County       16403
Vehicle Color         542695
dtype: int64


In [5]:
# violation code -> clean this first 
#     find lower threshold and remove violation tickets that dont lower threshold
# time -> format HHMM(A/P) -> use this to categorize time
# Vehicle Maker 
# Registration State
# Plate Type
# Vehicle Body Type
# Vehicle Color
#     find lower threshold by count and remove violation tickets that dont lower threshold
#     do by count 

# reason why we decide threshold by count
# the more people that has this description, the more relevant it is
# the less people that has this description, the less relevant it is 
# in terms of reflecting general population

# Hence
# reasoning to remove lower threshold
# this is because these descriptions do not reflect the overall population as the descriptions are not common
# lower threshold does not reflect 

# reason to keep upper threshold
# the more people that follows these descriptions, the more it reflects the general population
# hence, if we to remove upper threshold, it will essentially remove a huge chunk of our data that reflects general
# population

# for example
# if a violation code 'X' has only 1 violation 
# then it does not make sense to include 'X' since it does not reflect overall population
# and if a violation code 'Y' has 1000000
# then it does make sense to include Y

# in comparison to the total number of data
# a small portion of data has None values
# also in this context, we cannot use stuff we dont know as filling the none values will introduce inaccurate values
# so we could first drop Null values as it would not have a significant impact on the data

df = df.dropna()
print(df.isnull().sum())



Registration State    0
Plate Type            0
Violation Code        0
Vehicle Body Type     0
Vehicle Make          0
Violation Precinct    0
Violation Time        0
Violation County      0
Vehicle Color         0
dtype: int64


In [6]:
# approximately 500000 rows are removed mostly from color
print(df.shape)

(5562715, 9)


In [7]:
# inspect each column values to what needs further processing
print(df['Registration State'].unique())
# '99' do not look like a state so see how many vehicles are under this state
df = df[df['Registration State'] != '99']
print(df['Registration State'].unique())


['NY' 'NJ' 'PA' 'FL' 'OH' 'MA' 'NC' 'UT' '99' 'GA' 'TN' 'CA' 'CT' 'MS'
 'RI' 'TX' 'MN' 'AL' 'WA' 'ME' 'AR' 'IA' 'GV' 'OK' 'MI' 'DC' 'VA' 'VT'
 'ID' 'SC' 'AK' 'ON' 'MO' 'AZ' 'MD' 'LA' 'IN' 'NH' 'DE' 'OR' 'ND' 'NV'
 'QB' 'BC' 'NM' 'HI' 'IL' 'WI' 'MT' 'CO' 'MB' 'KY' 'PR' 'NS' 'WV' 'KS'
 'SD' 'NB' 'WY' 'NE' 'DP' 'AB' 'PE' 'SK' 'FO']
['NY' 'NJ' 'PA' 'FL' 'OH' 'MA' 'NC' 'UT' 'GA' 'TN' 'CA' 'CT' 'MS' 'RI'
 'TX' 'MN' 'AL' 'WA' 'ME' 'AR' 'IA' 'GV' 'OK' 'MI' 'DC' 'VA' 'VT' 'ID'
 'SC' 'AK' 'ON' 'MO' 'AZ' 'MD' 'LA' 'IN' 'NH' 'DE' 'OR' 'ND' 'NV' 'QB'
 'BC' 'NM' 'HI' 'IL' 'WI' 'MT' 'CO' 'MB' 'KY' 'PR' 'NS' 'WV' 'KS' 'SD'
 'NB' 'WY' 'NE' 'DP' 'AB' 'PE' 'SK' 'FO']


In [8]:
print(df['Plate Type'].unique())
# '999' do not look like a plate type so see how many under this and remove
df = df[df['Plate Type'] != '999']
print(df['Plate Type'].unique())
print(df['Plate Type'].value_counts())

['PAS' 'OMS' '999' 'OMT' 'SRF' 'COM' 'MED' 'ORG' 'MOT' 'TRL' 'HIS' 'SPO'
 'MCL' 'NYS' 'RGL' 'VPL' 'OMV' 'OML' 'VAS' 'HIR' 'PSD' 'HAM' 'TOW' 'SOS'
 'PHS' 'JSC' 'ITP' 'CMB' 'AGC' 'NLM' 'NYC' 'CSP' 'TRC' 'OMR' 'SEM' 'APP'
 'SRN' 'LMB' 'SCL' 'IRP' 'DLR' 'NYA' 'CHC' 'STA' 'TRA' 'LTR' 'RGC' 'ORC'
 'ARG' 'SPC' 'LMA' 'SUP' 'ATV' 'AYG' 'AMB' 'BOB' 'AGR' 'STG' 'CLG' 'CME'
 'LMC' 'CBS' 'JCA' 'MCD' 'CCK' 'CMH' 'PPH' 'OMF' 'THC' 'HSM' 'GSM' 'WUG'
 'USC' 'FAR' 'JWV' 'FPW' 'HOU' 'LUA' 'OMO']
['PAS' 'OMS' 'OMT' 'SRF' 'COM' 'MED' 'ORG' 'MOT' 'TRL' 'HIS' 'SPO' 'MCL'
 'NYS' 'RGL' 'VPL' 'OMV' 'OML' 'VAS' 'HIR' 'PSD' 'HAM' 'TOW' 'SOS' 'PHS'
 'JSC' 'ITP' 'CMB' 'AGC' 'NLM' 'NYC' 'CSP' 'TRC' 'OMR' 'SEM' 'APP' 'SRN'
 'LMB' 'SCL' 'IRP' 'DLR' 'NYA' 'CHC' 'STA' 'TRA' 'LTR' 'RGC' 'ORC' 'ARG'
 'SPC' 'LMA' 'SUP' 'ATV' 'AYG' 'AMB' 'BOB' 'AGR' 'STG' 'CLG' 'CME' 'LMC'
 'CBS' 'JCA' 'MCD' 'CCK' 'CMH' 'PPH' 'OMF' 'THC' 'HSM' 'GSM' 'WUG' 'USC'
 'FAR' 'JWV' 'FPW' 'HOU' 'LUA' 'OMO']
PAS    4516130
COM     551819
OMT     2093

In [9]:
print(df['Violation Code'].unique())
# all violation codes in data
# choose violations that most reflect general population
print(df['Violation Code'].value_counts())
# df violtion code where code not in code_set
# create a dataframe
# violation code : count
# 



[67 51 63 98 10 45 14 20 68 27 53 19 40 74 16 17 48 62 70 46 50 71 21 95
 24 41 99 75 83 18 37 38 78 97 61 96 52 13 11 39 54 49  5 31 60 90 91  6
 92 85 55 56  9 66  0 80 36 72 22 26  7 69 12 87 33 73 76 79 82 64 25  4
 47 42 35 84 32  8 77 58 23 65 34 89 57 81 44 59  1 29  3  2 30 93 94]
36    1954915
21     716783
38     343279
14     290820
20     244402
       ...   
34          3
57          3
81          1
93          1
94          1
Name: Violation Code, Length: 95, dtype: int64


In [10]:
print(df['Vehicle Body Type'].unique())
# choose body type that most reflect general population
print(df['Vehicle Body Type'].value_counts())


['SDN' 'SUBN' 'VAN' 'FOUR' 'P-U' 'LIMO' 'TWOD' 'CONV' 'SADO' 'BLAC' 'OE'
 'DELV' 'DUMP' 'TAXI' 'MERC' 'BMW' 'OUE' '4DSD' 'OLE' 'SPOR' 'ONE' 'TRUC'
 'OLNE' 'OLME' 'OIE' 'MCY' 'SUV' 'SEDN' 'TT' 'OTSE' 'ONL' 'CNV' 'PICK'
 'MOTO' 'ON' 'OME' 'BLK' 'NE' 'BUS' 'TRL' 'OTE' '2DSD' 'BUCK' 'OTLE'
 'TRLR' 'TRAC' 'I' 'HTCH' 'SEMI' 'OLSE' '4S' 'UT' 'COUP' 'H/WH' 'ATV'
 'UTIL' 'AMBU' 'LIM' 'CUST' 'SW' 'WAGN' '5D' '4D' 'SEDA' '2D' 'LL' 'Seda'
 '4H' 'SD' 'VN' 'HB' '2S' 'TK' 'PK' 'UV' 'Hatc' 'JP' 'SU' 'MP' 'MC' '4W'
 'HRSE' 'CV' '2DOO' 'UTPC' 'SV' 'LI' 'APUR' 'S' 'Coup' '2H' 'CG' 'CP'
 'H/IN' 'LM' '43' 'PV' 'WAGO' 'WG' '3D' 'ROVE' 'CN' 'PAS' 'TOW' 'EC'
 'MINI' 'Spor' 'PE' 'SULB' 'PKPC' 'BOAT' 'TR/C' 'FLAT' 'REFG' 'MOBL' 'N/S'
 'STAK' 'T/CR' 'TRAV' 'SWT' 'P/SH' 'MOPD' 'POLE' 'TRAI' 'TR' 'TRA' 'ACCO'
 'OLL' 'TLR' 'TOWT' 'FRTC' 'TRH' 'TRC' 'TRK' 'TRAT' 'TF' 'ALFA' 'CON' 'TL'
 'CXE' 'BT' 'LINC' 'BICK' 'POWE' 'MOT' 'HYBR' 'MOPE' 'BOX' 'PILO' 'MD'
 'EBIK' 'RANG' 'TRTR' 'E' 'MACK' 'COM' 'ANE' 'OINI' 'MEY' 'DEL

In [11]:
print(df['Vehicle Make'].unique(), len(df['Vehicle Make'].unique()))
# too many vehicle makers
# choose relevant vehicle makers that reflects general population

['TOYOT' 'JEEP' 'HYUND' ... 'RUGB' 'COU' 'MAGIR'] 1567


In [12]:
print(df['Violation Precinct'].unique(), len(df['Violation Precinct'].unique()))
# prob nothing to be done here

[ 10   1   5   6  45 100 101 122 123 120  84  75  42  46  44  41  52  40
 113  76 112  72  69 103 110 105  62  14  25 108  63  67  60  23 109 106
  48  47  43  90   4   9  66  32  83   0 114  49  78  64  13 115  94  18
 111  88  73  30  68  61  24  34  50  99  70 107 121  17  28 119  19  20
  33  71  77 102  79   7  81  39  80 104  26  56 170  74  11  65  21 116
 117   8 168  38  29 200  37  22 136  85  35  51 144 272  53 163  36 805
 174  87 161 164 118   3  15  89 166 127 803 203  57  91 808   2  96  95
 160  93  16  97  54  31 171  82 933  58] 136


In [13]:
print(df['Violation Time'].unique(), len(df['Violation Time'].unique()))

# remove rows of string length violation time less than 5
df = df[df['Violation Time'].apply(lambda x: len(str(x)) == 5)]
print(df.shape)
# too many time, standardize time

# Parts of the Day

# Morning     5 am to 12 pm (noon)

# Afternoon     12 pm to 5 pm

# Evening     5 pm to 9 pm

# Night         9 pm to 4 am

# create a new column "Violation Day" iterate through each row and 
violation_time = df["Violation Time"]

violation_day = []
start_morning = 5
end_morning = 12

start_noon = 12
end_noon = 5 + 12

start_evening = 5 + 12
end_evening = 9 + 12


for i, time in enumerate(violation_time):
    hour = int(time[0:2])
    # edge case
    if(hour == 12 and time[4] == 'A'):
        hour = 0
    elif(hour != 12 and time[4] == 'P'):
        hour = hour + 12

    if hour >= start_morning and hour < end_morning:
        violation_day.append("Morning")
    elif hour >= start_noon and hour < end_noon:
        violation_day.append("Afternoon")
    elif hour >= start_evening and hour < end_evening:
        violation_day.append("Evening")
    else:
        violation_day.append("Night")
        
print("done")    
df["Violation Day"] = violation_day
#drop violation column, no longer need it
df = df.drop(columns="Violation Time")
print(df["Violation Day"].value_counts())

['1037A' '1045A' '1116A' ... '0012P' '01 6A' '6020P'] 1545
(5538714, 9)
done
Morning      2448237
Afternoon    1837223
Evening       645292
Night         607962
Name: Violation Day, dtype: int64


In [14]:
print(df['Violation County'].unique(), len(df['Violation County'].unique()))
# violation county has same county but different abbreivations

county_mapping = {'NY': 'NY','BX':'BX', 'Q':'QNS', 'R':'R', 'K':'K', 'MS': 'MS', 'BK': 'BK',
           'QN': 'QNS', 'MN': 'MN', 'Kings': 'K',
           'Qns': 'QNS', 'Bronx': 'BX', 'Rich': 'R', 'ST':'ST', 'QNS':'QNS', 'ABX': 'ABX'}

df['Violation County'] = df['Violation County'].map(county_mapping)
print(df['Violation County'].unique())
print(df['Violation County'].value_counts())
#drop 'MS' and 'ABX'
df = df[df['Violation County'] != 'MS']
df = df[df['Violation County'] != 'ABX']



['NY' 'BX' 'Q' 'R' 'K' 'MS' 'BK' 'QN' 'MN' 'Kings' 'Qns' 'Bronx' 'Rich'
 'ST' 'QNS' 'ABX'] 16
['NY' 'BX' 'QNS' 'R' 'K' 'MS' 'BK' 'MN' 'ST' 'ABX']
QNS    1509564
NY     1316070
K       782040
BX      754488
BK      744460
MN      227268
ST      150381
R        54441
MS           1
ABX          1
Name: Violation County, dtype: int64


In [15]:
print(df['Violation County'].value_counts())
print(df['Vehicle Color'].unique(), len(df['Vehicle Color'].unique()))
# on closer inspection
# color abbreivation is not standardized
# so we see same colors with different abbreivation
# transform this

QNS    1509564
NY     1316070
K       782040
BX      754488
BK      744460
MN      227268
ST      150381
R        54441
Name: Violation County, dtype: int64
['BLK' 'GRAY' 'BLUE' 'WHITE' 'RED' 'GY' 'WHT' 'BLACK' 'GL' 'SILVE' 'GREY'
 'BL' 'TAN' 'WH' 'BLU' 'BRO' 'GRY' 'ORANG' 'PURPL' 'SLV' 'SIL' 'WT' 'ORG'
 'SUILV' 'BEIGE' 'BRN' 'GREEN' 'BK' 'GR' 'GOLD' 'W' 'NAVY' 'SILV' 'R' 'G'
 'WHI' 'BUR' 'BROW' 'B' 'BROWN' 'SI' 'RD' 'BKL' 'MAROO' 'SL' 'YLLW' 'GRV'
 'GRN' 'GLD' 'SV' 'SLVR' 'BURG' 'SW' 'TN' 'WHTE' 'BN' 'GRAN' 'BR' 'WHIT'
 'PINK' 'GARY' 'GN' 'BRW' 'M' 'BEGE' 'Gray' 'YELLO' 'PURP' 'BLCK' 'WTE'
 'OR' 'T' 'BLA' 'SILVR' 'BW' 'BLAAC' 'BLACJ' 'MRN' 'PK' 'YEL' '.' 'SN'
 'YE' 'BKUE' 'RM' 'MARON' 'GKD' 'BG' 'LTGY' 'YELL' 'BLC' 'OLIV' 'WI'
 'BLAC' 'GRA' 'BCK' 'TEAL' 'BLAK' 'GLN' 'BWN' 'GRE' 'SIVR' 'BIGE' 'BURGU'
 'ON' 'GD' 'MN' 'PR' 'BRWN' 'PW' 'BURGA' 'BLE' 'NLK' 'SLIVE' 'SILVA' 'MR'
 'GREN' 'WHE' 'DK/GY' 'BLGY' 'LT/GR' 'DKRD' 'YW' 'DKGY' 'LTBK' 'LT/GY'
 'BL/BK' 'DK/BL' 'WHTN' 'DK/PR' 'DKGR' 'WHB

In [16]:
# transformation to be done
# select data that reflects general population
# this includes
# plate type
# violation type
# vehicle maker
# color -> choose color that we could easily translate
# 
df['Vehicle Color'].replace(['BLK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['GRAY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['BLUE'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['WHITE'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['RED'], ['RED'], inplace=True)
df['Vehicle Color'].replace(['GY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['WHT'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BLACK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['GL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SILVE'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['GREY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['BL'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['TAN'], ['TAN'], inplace=True)
df['Vehicle Color'].replace(['WH'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BLU'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['BRO'], ['BROW'], inplace=True)
df['Vehicle Color'].replace(['GRY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['ORANG'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['PURPL'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['SLV'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['SIL'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['WT'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['ORG'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['SUILV'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['BEIGE'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['BRN'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['GREEN'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['BK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['GR'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['GOLD'], ['GOLD'], inplace=True)
df['Vehicle Color'].replace(['W'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['NAVY'], ['NAVY'], inplace=True)
df['Vehicle Color'].replace(['SILV'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['R'], ['RED'], inplace=True)
df['Vehicle Color'].replace(['G'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['WHI'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BUR'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['BROW'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['B'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['BROWN'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['SI'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['RD'], ['RED'], inplace=True)
df['Vehicle Color'].replace(['BKL'], ['BLACL'], inplace=True)
df['Vehicle Color'].replace(['MAROO'], ['MAROO'], inplace=True)
df['Vehicle Color'].replace(['SL'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['YLLW'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['GRV'], ['v'], inplace=True)
df['Vehicle Color'].replace(['GRN'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['GLD'], ['GOLD'], inplace=True)
df['Vehicle Color'].replace(['SLVR'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['BURG'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['SW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TN'], ['TAN'], inplace=True)
df['Vehicle Color'].replace(['WHTE'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BN'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['GRAN'], ['GRAN'], inplace=True)
df['Vehicle Color'].replace(['BR'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['WHIT'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['PINK'], ['PINK'], inplace=True)
df['Vehicle Color'].replace(['GARY'], ['GARY'], inplace=True)
df['Vehicle Color'].replace(['GN'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['BRW'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['M'], ['MAROON'], inplace=True)
df['Vehicle Color'].replace(['BEGE'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['Gray'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['YELLO'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['PURP'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['BLCK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['WTE'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['OR'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['T'], ['TAN'], inplace=True)
df['Vehicle Color'].replace(['BLA'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['SILVR'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['BW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLAAC'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['BLACJ'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['MRN'], ['MAROO'], inplace=True)
df['Vehicle Color'].replace(['PK'], ['PINK'], inplace=True)
df['Vehicle Color'].replace(['YEL'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['.'], ['N/A'], inplace=True)
df['Vehicle Color'].replace(['SN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YE'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['BKUE'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['RM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MARON'], ['MAROON'], inplace=True)
df['Vehicle Color'].replace(['GKD'], ['GOLD'], inplace=True)
df['Vehicle Color'].replace(['BG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YELL'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['BLC'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['OLIV'], ['OLIVE'], inplace=True)
df['Vehicle Color'].replace(['WI'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BLAC'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['GRA'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['BCK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['TEAL'], ['TEAL'], inplace=True)
df['Vehicle Color'].replace(['BLAK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['GLN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BWN'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['GRE'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['SIVR'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['BIGE'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['BURGU'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['ON'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['GD'], ['GOLD'], inplace=True)
df['Vehicle Color'].replace(['MN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PR'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['BRWN'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['PW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BURGA'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['BLE'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['NLK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['SLIVE'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['SILVA'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['MR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GREN'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['WHE'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['DK/GY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LT/GR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YW'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['DKGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LT/GY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BL/BK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DK/BL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHTN'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['DK/PR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH/BK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['NOCL'], ['N/A'], inplace=True)
df['Vehicle Color'].replace(['LT/BL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GLGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH/BL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SLK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Black'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['BKGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKMR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYGY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['UNKNO'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['LTGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YWBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['OTHER'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Silve'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['GRTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Unkno'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BGE'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['WHGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYYA'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TNGR'], ['TANGERINE'], inplace=True)
df['Vehicle Color'].replace(['GLGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MRBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DARK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['TNGY'], ['TANGERINE'], inplace=True)
df['Vehicle Color'].replace(['BKBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['White'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['TNRD'], ['TANGERINE'], inplace=True)
df['Vehicle Color'].replace(['ALUMI'], ['ALUMI'], inplace=True)
df['Vehicle Color'].replace(['RDBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MAR'], ['MAROON'], inplace=True)
df['Vehicle Color'].replace(['BKRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['CHROM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Red'], ['RED'], inplace=True)
df['Vehicle Color'].replace(['TAUPE'], ['TAUPE'], inplace=True)
df['Vehicle Color'].replace(['GRRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYOR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKX'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ONG'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['DKPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LIGHT'], ['LIGHT'], inplace=True)
df['Vehicle Color'].replace(['BRZ'], ['BRONZE'], inplace=True)
df['Vehicle Color'].replace(['`'], ['N/A'], inplace=True)
df['Vehicle Color'].replace(['GRBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRGR'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['WHKMK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TNPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Blue'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['GLBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MRGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Gold'], ['GOLD'], inplace=True)
df['Vehicle Color'].replace(['BRGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORBK'], ['ORBK'], inplace=True)
df['Vehicle Color'].replace(['MULTI'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['CRM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDMR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BL.'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['PRWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKOR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKI'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MRTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['COM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYQW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TNWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TURQU'], ['TURQUOISE'], inplace=True)
df['Vehicle Color'].replace(['PLE'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH1'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BRGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Maroo'], ['MAROO'], inplace=True)
df['Vehicle Color'].replace(['RDBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRCDF'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKBK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['TNBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PKGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Tan'], ['TAN'], inplace=True)
df['Vehicle Color'].replace(['BKL.['], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['LTG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYC'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYMR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SLVER'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['DGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY`'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['RDV.'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MRRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRONZ'], ['BRONZE'], inplace=True)
df['Vehicle Color'].replace(['AME'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TNGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['NOC'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DK/'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['DKB'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['LT/'], ['LIGHT'], inplace=True)
df['Vehicle Color'].replace(['BKG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LAVEN'], ['LAVENDAR'], inplace=True)
df['Vehicle Color'].replace(['WHB'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY/'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['GYB'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTB'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['NO'], ['N/A'], inplace=True)
df['Vehicle Color'].replace(['WH/'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['GRT'], ['GRTN'], inplace=True)
df['Vehicle Color'].replace(['DKR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BL/'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['RDG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BK/'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['LTP'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHO'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['DKP'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TNG'], ['TANGERINE'], inplace=True)
df['Vehicle Color'].replace(['BLT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLB'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRB'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['G'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['ORA'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['ORGN'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['HYUND'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['YELLW'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['YLW'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['BIEGE'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['TRQ'], ['TURQUOISE'], inplace=True)
df['Vehicle Color'].replace(['PUR'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['BUE'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['PIBK'], ['PINK'], inplace=True)
df['Vehicle Color'].replace(['U'], ['UKNOWN'], inplace=True)
df['Vehicle Color'].replace(['PNK'], ['PINK'], inplace=True)
df['Vehicle Color'].replace(['PINE'], ['PINE'], inplace=True)
df['Vehicle Color'].replace(['YLL'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['.'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['..'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['TAM'], ['TANGERINE'], inplace=True)
df['Vehicle Color'].replace(['GREE'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['E'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['HREEN'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['Y'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['MIX'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHIE'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['OM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BACK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace([''], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['.'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BIK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['B'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['LAC'], ['LAVENDAR'], inplace=True)
df['Vehicle Color'].replace(['WHIYE'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['HU'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['SIC'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['GOL'], ['GOLD'], inplace=True)
df['Vehicle Color'].replace(['BKACK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['SVE'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['WHU'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['MC'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['RO'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['YU'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['LT/BR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PRPLE'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['DK/RD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YL'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['PRL'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['PRPL'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['NO'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['CL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BL/WH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RD/GY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BUG'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['DK/BK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHTIE'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['GRTEY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['BGI'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['GEY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['GT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY/BK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['S'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['REDBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['OR/WH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['UNK'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['WHOR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DK/GR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LAV'], ['LAVENDAR'], inplace=True)
df['Vehicle Color'].replace(['GYWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH/OR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YWBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['CPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH/PI'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKMR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLUE,'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['PEWTE'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PRGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDRD'], ['RED'], inplace=True)
df['Vehicle Color'].replace(['?:{'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['YWRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GLRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YWWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TNBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHPK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['COPPE'], ['COPPER'], inplace=True)
df['Vehicle Color'].replace(['RDOR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MUL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHI-T'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['WHWH'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['DKWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['XXX'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BKH'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BLKTA'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYBN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH+'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['LTWH'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BRU76'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['I'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['GLBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['CREAM'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['YWGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRYGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['OIU'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['Brown'], ['BROWN'], inplace=True)
df['Vehicle Color'].replace(['GRWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SIL/B'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TPE'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['YW'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['Dark'], ['DARK'], inplace=True)
df['Vehicle Color'].replace(['MRBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['REDRE'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['WHYW'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['GG'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['WHN'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['MROR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BK+'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['LGT'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['TEA'], ['TEA'], inplace=True)
df['Vehicle Color'].replace(['BLYW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PRRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKYW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BL6'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['Q'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['GYHNU'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['LTYW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLMR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YWGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PKWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYX'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['Other'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHMR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BK,'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['BKOR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRY/B'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RD/'], ['RED'], inplace=True)
df['Vehicle Color'].replace(['GRW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GR/'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['TN/'], ['TAN'], inplace=True)
df['Vehicle Color'].replace(['RDT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRG'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['BRT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Green'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['GYG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RUST'], ['BRONZE'], inplace=True)
df['Vehicle Color'].replace(['.'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['K'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['WHHIT'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['NVY'], ['NAVY'], inplace=True)
df['Vehicle Color'].replace(['B'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['GRET'], ['GREEN'], inplace=True)
df['Vehicle Color'].replace(['GRL'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BALCK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['STI'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SIV'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['.'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['-'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['QUEEN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WJ'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['SIK'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['BLV'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['PIK'], ['PINK'], inplace=True)
df['Vehicle Color'].replace(['BURUN'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['WHK'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['WE'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['ED'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SILVV'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['SVR'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['LT'], ['LIGHT'], inplace=True)
df['Vehicle Color'].replace(['HRY'], ['OTHER'], inplace=True)

df['Vehicle Color'].replace(['..K'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['COL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LT/BK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BK/GY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY/TN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['NC'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['LT/TN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH/GL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLGL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYYW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['NOCLB'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BK.'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['TNBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY42-'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BK0-'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKNO'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTMR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRMR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['AMETH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['OO'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['WH.0'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['GLB'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BU'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['BLACL'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['SLI'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['RL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BURGD'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['HULT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRYSL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BURGN'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['C'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SILE'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['-'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['FRAY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['..'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['OG'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['WTH'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['WHJT'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['MA'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRIGE'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['WHY'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['ORNG'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['ORN'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['FRY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GAY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['PRP'], ['PRUPLE'], inplace=True)
df['Vehicle Color'].replace(['BL'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['BK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['GNE'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BJ'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['HTAY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['BLUEB'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['SIVER'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['BORG'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['GGRAY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['N'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BX'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['BL'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['K'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SLR'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['GTY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BK/BL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BEKE'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['SILER'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['WH/TN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHITW'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BBLK'], ['BLACK'], inplace=True)

df['Vehicle Color'].replace(['A'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['C'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLUR'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['PR/WH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BL/GY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DK/BR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH/RD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY/BR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY/GL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TN/BR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['D'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHGRN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RD,.;'], ['RED'], inplace=True)
df['Vehicle Color'].replace(['GY.'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['GLOR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYJUU'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['+++++'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['LTRD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['//'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['NOCLR'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['Light'], ['LIGHT'], inplace=True)
df['Vehicle Color'].replace(['GLTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MRGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['O'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['A'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY1'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['BLUGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['0'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH-'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BK/TN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YW0'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['BRBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH`'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['PRBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLOR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MRWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TNMR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RUNNE'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHV'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['Yello'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['TAN/B'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORGR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRQWW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYII'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['LTPK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY245'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BK333'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TNTN'], ['TAN'], inplace=True)
df['Vehicle Color'].replace(['GLWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['X'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TNOR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKPK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYA'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['AZ'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['N'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['NOCLG'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BKJ'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['PKPR'], ['BKPR'], inplace=True)
df['Vehicle Color'].replace(['MARO'], ['MAROO'], inplace=True)
df['Vehicle Color'].replace(['WHGY8'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['LTPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WITH'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['.N.D'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BURGY'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['DARKB'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BERG'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['BY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRAP'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['B'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['L'], ['LIGHT'], inplace=True)
df['Vehicle Color'].replace(['BUS'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WHTW'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['ORAN'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['SVL'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['BL/GR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YWTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKC'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['TN,1;'], ['TAN'], inplace=True)
df['Vehicle Color'].replace(['GLBL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY3'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['RDPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Bronz'], ['BRONZE'], inplace=True)
df['Vehicle Color'].replace(['TNYW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PRW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['-M.'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BEIG'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['PARK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YELOW'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['SLIV'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['WORKH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BUY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRTAY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['GRU'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ERD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['VAN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TA'], ['TAN'], inplace=True)
df['Vehicle Color'].replace(['TRY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['B'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['K'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BOLD'], ['GOLD'], inplace=True)
df['Vehicle Color'].replace(['QHITE'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['WTC'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDE'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['OTH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GR/PR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BL/GL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY/BL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RD/OR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['Purpl'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['GRBR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PKGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MRBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['D.GRE'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDNL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH3'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['GROR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GO'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLGY0'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GYXV'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLUN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH-+/'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['GR0'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PRYW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH/GY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['I'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MRG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MO'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['S'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['ILV'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['GRAG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DILVE'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['NLUE'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['MULT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SLVRE'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['GGREY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['PL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DIL'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['KX'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BE'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PUPLE'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['/BLK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['WHITR'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BOMBY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GERY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['TN/GR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MR/TN'], ['MRTN'], inplace=True)
df['Vehicle Color'].replace(['BK1'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['DKPK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['WH,'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['BR0'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RD`'], ['RED'], inplace=True)
df['Vehicle Color'].replace(['BLP'], ['BLPR'], inplace=True)
df['Vehicle Color'].replace(['TRED'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['CHAMP'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['TRLR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RE'], ['RED'], inplace=True)
df['Vehicle Color'].replace(['SILVB'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['BERGE'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['NLACK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['OAGE'], ['ORANGE'], inplace=True)
df['Vehicle Color'].replace(['OT'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['AW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['P'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['--N'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['GW'], ['GRWH'], inplace=True)
df['Vehicle Color'].replace(['WHUIT'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['MV'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['.UN'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BLD'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GE'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['GRAYY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['FREIG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['PINKT'], ['PINK'], inplace=True)
df['Vehicle Color'].replace(['ISUZU'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SILU'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace(['DARKG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BRG'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['...'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['GTAY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['BLKWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BEI'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['GRI'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BEG'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['WTI'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['NAVI'], ['NAVY'], inplace=True)
df['Vehicle Color'].replace(['WL'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['SLU'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BGDY'], ['BURGUN'], inplace=True)
df['Vehicle Color'].replace(['GHY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['CHY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['PIN'], ['PINK'], inplace=True)
df['Vehicle Color'].replace(['GRNGY'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DKYW'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['YWOR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['DRK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKZ'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['CAM'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['CC'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GY][]'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BGE/G'], ['BEIGE'], inplace=True)
df['Vehicle Color'].replace(['BLF'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['RDGUI'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BK3'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['BK0'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['. ..'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace([' .'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['B LAC'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['.  K'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['NO CL'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['NVY B'], ['NAVY'], inplace=True)
df['Vehicle Color'].replace(['. _'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace([' ..K'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['WH I'], ['WHITE'], inplace=True)
df['Vehicle Color'].replace(['S ILV'], ['SILVER'], inplace=True)
df['Vehicle Color'].replace([' A C'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['GY N'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['  BLK'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['B L'], ['BLUE'], inplace=True)
df['Vehicle Color'].replace(['GY N'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['B K'], ['BLACK'], inplace=True)
df['Vehicle Color'].replace(['BK0-\''], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['. -'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['GY G'], ['UNKNOWN'], inplace=True)
df['Vehicle Color'].replace(['BL BK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['v'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['SV'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace([' OIU'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace([' YW'], ['YELLOW'], inplace=True)
df['Vehicle Color'].replace(['MAROO'], ['MAROON'], inplace=True)
df['Vehicle Color'].replace(['GRTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BL K'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BKPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['BLPR'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GRWH'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['MRTN'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['ORBK'], ['OTHER'], inplace=True)
df['Vehicle Color'].replace(['GARY'], ['GRAY'], inplace=True)
df['Vehicle Color'].replace(['PRUPLE'], ['PURPLE'], inplace=True)
df['Vehicle Color'].replace(['UKNOWN'], ['UNKNOWN'], inplace=True)
print(df['Vehicle Color'].value_counts())

BLACK        1718897
WHITE        1489104
GRAY         1399612
RED           295506
BROWN         121940
OTHER         100738
GREEN          93686
BLUE           90071
SILVER         89748
TAN            51598
YELLOW         45498
ORANGE         18864
GOLD            7788
PURPLE          5616
LIGHT           3209
UNKNOWN         2860
MAROON          1095
N/A              841
BEIGE            603
LAVENDAR         474
PINK             356
BRONZE           199
BURGUN           140
TANGERINE        106
ALUMI             60
TURQUOISE         28
TAUPE             27
TEA               17
TEAL              12
NAVY              11
COPPER             4
GRAN               1
OLIVE              1
PINE               1
DARK               1
Name: Vehicle Color, dtype: int64


In [21]:
# transformation to be done
# select data that reflects general population
# this includes
# plate type
# violation type
# vehicle maker
# color -> choose color that we could easily translate
# 
print(df['Vehicle Color'].value_counts())
print(df['Vehicle Color'].unique(), len(df['Vehicle Color'].unique()))

BLACK        1718897
WHITE        1489104
GRAY         1399612
RED           295506
BROWN         121940
OTHER         100738
GREEN          93686
BLUE           90071
SILVER         89748
TAN            51598
YELLOW         45498
ORANGE         18864
GOLD            7788
PURPLE          5616
LIGHT           3209
UNKNOWN         2860
MAROON          1095
N/A              841
BEIGE            603
LAVENDAR         474
PINK             356
BRONZE           199
BURGUN           140
TANGERINE        106
ALUMI             60
TURQUOISE         28
TAUPE             27
TEA               17
TEAL              12
NAVY              11
COPPER             4
GRAN               1
OLIVE              1
PINE               1
DARK               1
Name: Vehicle Color, dtype: int64
['BLACK' 'GRAY' 'BLUE' 'WHITE' 'RED' 'OTHER' 'SILVER' 'TAN' 'BROWN'
 'ORANGE' 'PURPLE' 'BEIGE' 'GREEN' 'GOLD' 'NAVY' 'BURGUN' 'MAROON'
 'YELLOW' 'GRAN' 'PINK' 'N/A' 'OLIVE' 'TEAL' 'UNKNOWN' 'TANGERINE' 'ALUMI'
 'TAUPE' 'LIGHT' 'BRO

In [18]:
# write a funciton to get the counts + quartile + iqr
# find lower threshold remove it
# apply function on each of the data
# analyze data
